# CSM ISD
The community sensor model utilizes the concept of image support data to disentagle the process of collecting and collating a priori sensor information from the process of determining where the image is in relation to some reference frame.  In this notebook, we leverage the PVL and SpiceyPy Python libraries to generate two CSM compliant ISD objects.

In [1]:
import json
import spiceypy as spice
import pvl
import cycsm as csm  # This is the USGS cython wrapper to the C++ CSM

In [2]:
# Utility Func for working with PVL
def find_in_dict(obj, key):
    """
    Recursively find an entry in a dictionary

    Parameters
    ----------
    obj : dict
          The dictionary to search
    key : str
          The key to find in the dictionary

    Returns
    -------
    item : obj
           The value from the dictionary
    """
    if key in obj:
        return obj[key]
    for k, v in obj.items():
        if isinstance(v,dict):
            item = find_in_dict(v, key)
            if item is not None:
                return item

# MDIS WAC 
We start with the MDIS WAC camera and generate a JSON ISD string that can be used to create an ISD.  This ISD utilizes the NAIF Spice Library and SPICE kernels.

First, we check for the kernels that will be furnished.

In [12]:
ikid = 236800
msgr = "/data/isis3/data/messenger/"  # This is the standard ISIS3 kernel location
# Load kernels same order ISIS Spice::init() does
# Frame
# TargetAttitudeShape
#spice.furnsh(msgr+"kernels/pck/pck00010_msgr_v23.tpc")  # Not shipping with ISIS (why?)
spice.furnsh(msgr + 'kernels/pck/pck00010_MSGR_v10.tpc')  # Publicly available
# Instrument
spice.furnsh(msgr+"kernels/ik/msgr_mdis_v160.ti")
# InstrumentAddendum
spice.furnsh(msgr+"kernels/iak/mdisAddendum009.ti")
# LeapSecond
spice.furnsh("/data/isis3/data/base/kernels/lsk/naif0012.tls")
# SpacecraftClock
spice.furnsh(msgr+"kernels/sclk/messenger_2548.tsc")
# Extra
# TargetPosition
spice.furnsh(msgr+"kernels/tspk/de423s.bsp")
# InstrumentPointing
spice.furnsh(msgr+"kernels/ck/msgr20150409.bc")
spice.furnsh(msgr+"kernels/ck/msgr20150410.bc")
spice.furnsh(msgr+"kernels/ck/msgr20150411.bc")
spice.furnsh(msgr+"kernels/ck/msgr20150412.bc")
spice.furnsh(msgr+"kernels/ck/msgr20150413.bc")
spice.furnsh(msgr+"kernels/ck/msgr20150414.bc")
spice.furnsh(msgr+"kernels/ck/msgr20150415.bc")
spice.furnsh(msgr+"kernels/ck/msgr20150416.bc")
spice.furnsh(msgr+"kernels/ck/1072683119_1965_mdis_atthist.bc")
spice.furnsh(msgr+"kernels/ck/1072716050_291010_mdis_pivot_pvtres.bc")
spice.furnsh(msgr+"kernels/fk/msgr_v231.tf")
# InstrumentPosition
spice.furnsh(msgr+"kernels/spk/msgr_20040803_20150430_od431sc_2.bsp")

ISD as a CSM ISD Object

In [14]:
# Create the ISD object
isd = csm.isd.Isd()

# Load information from the IK kernel
isd.addparam('focal_length',spice.gdpool('INS-{}_FOCAL_LENGTH'.format(ikid), 0, 1))
isd.addparam('focal_length_epsilon', spice.gdpool('INS-{}_FL_UNCERTAINTY'.format(ikid), 0, 1))

nlines = spice.gipool('INS-{}_PIXEL_LINES'.format(ikid), 0, 1)
nsamples = spice.gipool('INS-{}_PIXEL_SAMPLES'.format(ikid), 0, 1)
isd.addparam('nlines', nlines)
isd.addparam('nsamples', nsamples)
isd.addparam('original_half_lines', nlines / 2.0)
isd.addparam('original_half_samples', nsamples / 2.0)
isd.addparam('pixel_pitch', spice.gdpool('INS-{}_PIXEL_PITCH'.format(ikid), 0, 1))
isd.addparam('ccd_center', spice.gdpool('INS-{}_CCD_CENTER'.format(ikid), 0, 2))
isd.addparam('ifov', spice.gdpool('INS-{}_IFOV'.format(ikid), 0, 1))
isd.addparam('boresight', spice.gdpool('INS-{}_BORESIGHT'.format(ikid), 0, 3))
isd.addparam('transx', spice.gdpool('INS-{}_TRANSX'.format(ikid), 0, 3))
isd.addparam('transy', spice.gdpool('INS-{}_TRANSY'.format(ikid), 0, 3))
isd.addparam('itrans_sample', spice.gdpool('INS-{}_ITRANSS'.format(ikid), 0, 3))
isd.addparam('itrans_line', spice.gdpool('INS-{}_ITRANSL'.format(ikid), 0, 3))
isd.addparam('odt_x', spice.gdpool('INS-{}_OD_T_X'.format(ikid), 0, 10))
isd.addparam('odt_y', spice.gdpool('INS-{}_OD_T_Y'.format(ikid), 0, 10))
isd.addparam('starting_detector_sample', spice.gdpool('INS-{}_FPUBIN_START_SAMPLE'.format(ikid), 0, 1))
isd.addparam('starting_detector_line', spice.gdpool('INS-{}_FPUBIN_START_LINE'.format(ikid), 0, 1))

# Distortion Model

In [15]:
def distort_focal_length(coeffs, t):
    """
    Compute the distorted focal length
    
    Parameters
    ----------
    coeffs : iterable
             of coefficient values
    t : float
        temperature in C
        
    Returns
    -------
    focal_length : float
                   the temperature adjusted focal length
    """
    focal_length = coeffs[0]
    for i in range(1, len(coeffs[1:])):
        focal_length += coeffs[i]*t**i
    return focal_length

# Image Information
Load the image specific information from an ISIS3 cub PVL header.  It would be possible to use a cube that had already been spiceinit(ed) and just grab the appropriate kernel paths from there.

In [21]:
header = pvl.load("../tests/data/CW1071364100B_IU_5.cub")

isd.addparam('instrument_id',find_in_dict(header, 'InstrumentId'))
isd.addparam('spacecraft_name',find_in_dict(header, 'SpacecraftName'))
isd.addparam('target_name', find_in_dict(header, 'TargetName'))
filter_num = find_in_dict(header, 'OriginalFilterNumber')
ikid += filter_num

# Get the radii from SPICE
rad = spice.bodvrd(isd.param('target_name'), 'RADII', 3)
radii = rad[1]
isd.addparam('semi_major_axis', rad[1][0])
isd.addparam('semi_minor_axis', rad[1][1])


# Get temperature from SPICE and adjust focal length
spice.gdpool('INS-{}_FOCAL_LENGTH'.format(ikid), 0, 1)
temp_coeffs = spice.gdpool('INS-{}_FL_TEMP_COEFFS'.format(ikid), 0, 6)
temp = find_in_dict(header, 'FocalPlaneTemperature').value
isd.addparam('focal_length', distort_focal_length(temp_coeffs, temp))

In [23]:
# Here convert the sclock
sclock = find_in_dict(header, 'SpacecraftClockCount')
exposure_duration = find_in_dict(header, 'ExposureDuration')
exposure_duration = exposure_duration.value * 0.001  # Scale to seconds

# Get the instrument id, and, since this is a framer, set the time to the middle of the exposure
spacecraft_id = spice.bods2c('MESSENGER')
et = spice.scs2e(spacecraft_id, sclock)
et += (exposure_duration / 2.0)

isd.addparam('ephemeris_time', et)

In [26]:
loc, _ = spice.spkpos(isd.param('target_name'), isd.param('ephemeris_time'), 'IAU_MERCURY', 'LT+S', 'MESSENGER')
isd.addparam('x_sensor_origin', loc[0]  * -1000)
isd.addparam('y_sensor_origin', loc[1]  * -1000)
isd.addparam('z_sensor_origin', loc[2]  * -1000)

In [28]:
camera2bodyfixed = spice.pxform('MSGR_MDIS_WAC', 'IAU_MERCURY', isd.param('ephemeris_time'))
camopk = spice.m2eul(camera2bodyfixed, 3, 2, 1)

isd.addparam('omega',camopk[2])
isd.addparam('phi', camopk[1])
isd.addparam('kappa', camopk[0])

In [29]:
# Get the sun's position relative to a Mercury-fixed frame.
target = "SUN"
et =  isd.param('ephemeris_time')
reference_frame = "IAU_MERCURY"
light_time_correction = "LT+S"
observer = "MERCURY"

sun_state, lt = spice.spkezr(target,
                             et,
                             reference_frame,
                             light_time_correction,
                             observer)

# Convert to meters
isd.addparam('x_sun_position', sun_state[0] * 1000)
isd.addparam('y_sun_position', sun_state[1] * 1000)
isd.addparam('z_sun_position', sun_state[2] * 1000)

# Done
This is a CSM compliant ISD object.  It would now be possible to generate a CSM compliant MDIS WAC camera using the USGS CSM.  We do not do this in this notebook.